# 理解全连接层  

CSY 2019-12-26  

### 关于paddle.fluid.data()    
https://www.paddlepaddle.org.cn/documentation/docs/zh/api_cn/fluid_cn/data_cn.html


paddle.fluid.data()是一个OP（算子），作用就是创建一个全局变量，可供计算图中的算子访问，可作为占位符用于数据输入。可以视为输入层或data层?    
name 是paddle.fluid.data()创建的全局变量的名字,是输入层输出的前缀标识。   
shape 声明了paddle.fluid.data()创建的全局变量的维度信息。  
shape中的None 表示不确定该维的元素数量，待程序执行中确定。  
shape中的-1 只能在shape的最前面，表示可以适应任何 batch size  
dtype 是paddle.fluid.data()创建的全局变量的数据类型，支持 bool,float16,float32,float64,int8,int16,int32,int64。  
用户 feed 的数据必须与 paddle.fluid.data() 创建的变量具有相同的 shape  
虽然feed的数据，其类型是unsigned Byte,但softmax 回归是要进行浮点运算的，所以数据类型都转换成了float32

### 关于paddle.fluid.layers.fc()  
https://www.paddlepaddle.org.cn/documentation/docs/zh/api_cn/layers_cn/fc_cn.html

paddle.fluid.layers.fc()是一个OP，作用就是建立一个全连接层。为每个输入的Tensor创建一个权重变量，即一个从每个输入单元到每个输出单元的全连接权重矩阵。  
FC层将每个输入Tensor和其对应的权重(weights)相乘得到shape为 [M,size] 输出Tensor，其中 M 为batch_size大小。如果有多个输入Tensor，则多个shape为 [M,size] 的Tensor计算结果会被累加起来，作为最终输出。
仅有一个全连接层的网络，全连接层的input数据通常由paddle.fluid.data()产生。  

例1 参见《从简单的线性方程开始了解Paddle Fluid的全连接层》  
https://mp.csdn.net/postedit/103728321

In [63]:
import paddle.fluid as fluid
import numpy

In [73]:
#准备原始数据
input_data=numpy.array([[0.00632,18,2.31,0.538]]).astype('float32')
print(input_data.shape) # 查看原始数据的shape=[4]

(1, 4)


神经网络的每一层都有输入端和输出端。  
本例只有输入层和全连接层。  
其中输入层的输入端子名为datax,输出端子名为x。  
全连接层的输入端子与输入层的x相连，输出端子名为y。

In [74]:
#定义输入层
x=fluid.data(name="datax",shape=[-1,4],dtype='float32') 
#定义全连接层
y=fluid.layers.fc(input=x,size=1,act=None)

In [75]:
#定义执行器：cpu版
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program()) # 运行默认的启动程序

[]

In [76]:
# 运行并打印结果
out = exe.run(feed={'datax':input_data},fetch_list=[y]) 
print(out)

/home/csy/anaconda3/lib/python3.7/site-packages/paddle/fluid/executor.py:774: UserWarning: The following exception is not an EOF exception.
  "The following exception is not an EOF exception.")


EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::string paddle::platform::GetTraceBackString<std::string const&>(std::string const&, char const*, int)
1   paddle::platform::EnforceNotMet::EnforceNotMet(std::string const&, char const*, int)
2   paddle::operators::MulOp::GetExpectedKernelType(paddle::framework::ExecutionContext const&) const
3   paddle::framework::OperatorWithKernel::ChooseKernel(paddle::framework::RuntimeContext const&, paddle::framework::Scope const&, boost::variant<paddle::platform::CUDAPlace, paddle::platform::CPUPlace, paddle::platform::CUDAPinnedPlace, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_

In [22]:
#准备原始数据
data1=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32')
print(data1.shape,data1.data)
#定义输入层
y=fluid.layers.data(name="feed_data",shape=[1],dtype='float32') 
z = fluid.layers.fc(input=y,size=1,act=None) #全连接层直接与输入层相连

(4, 1) <memory at 0x7f87ed7cd1f8>


In [4]:
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())

[]

In [8]:
out = exe.run(feed={'feed_data':data1},fetch_list=[y,z]) # 输入层的feed_data端喂入data1数据。返回fetch_list指定的数据
out

[array([[1.],
        [2.],
        [3.],
        [4.]], dtype=float32), array([[0.8370477],
        [1.6740954],
        [2.5111432],
        [3.3481908]], dtype=float32)]